In [ ]:
%pip install -r requirements.txt

In [2]:
# Core dependencies for agent functionality
from xpander_sdk import Agent, XpanderClient, LLMProvider, ToolCallResult, Tokens, LLMTokens
from openai import OpenAI
import os
import time
from datetime import datetime
from dotenv import load_dotenv

# PDF generation dependencies
from reportlab.lib.pagesizes import A4
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.enums import TA_LEFT
from reportlab.lib.units import cm
import tempfile
import shutil

# Load environment variables and initialize clients
load_dotenv()
xpander_client = XpanderClient(api_key=os.environ['XPANDER_API_KEY'])
openai_client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])

In [3]:
def export_meeting_schedule_pdf(meetings: list) -> str:
    """
    Generate a clean, well-formatted PDF agenda for weekly meetings and save it to the user's Downloads folder.
    Returns the full path to the saved PDF.
    
    Args:
        meetings (list): List of meeting dictionaries containing:
            - title (str): Meeting title
            - start_time (str): ISO 8601 formatted start time
            - end_time (str): ISO 8601 formatted end time
            - location (str, optional): Meeting location
            - participants (list, optional): List of participant names/emails
    """
    if not meetings:
        return "No meetings provided."

    # Sort meetings by start time for chronological order
    meetings.sort(key=lambda m: m.get("start_time", ""))

    # Create temporary PDF file
    temp_pdf = tempfile.NamedTemporaryFile(delete=False, suffix=".pdf")
    doc = SimpleDocTemplate(temp_pdf.name, pagesize=A4)
    elements = []

    # Add title and styling
    styles = getSampleStyleSheet()
    title = Paragraph("<b>Weekly Meeting Agenda</b>", styles["Title"])
    elements.append(title)
    elements.append(Spacer(1, 12))

    # Define paragraph style for table cells
    cell_style = ParagraphStyle(name='Cell', fontSize=10, leading=12, alignment=TA_LEFT)

    # Prepare table data with headers
    data = [["Date", "Time", "Title", "Location", "Participants"]]

    # Process each meeting
    for meeting in meetings:
        try:
            # Format date and time
            start_dt = datetime.fromisoformat(meeting["start_time"])
            end_dt = datetime.fromisoformat(meeting["end_time"])
            date = start_dt.strftime("%A, %b %d")
            time = f"{start_dt.strftime('%H:%M')} - {end_dt.strftime('%H:%M')}"
        except Exception:
            date = time = "Invalid"

        # Format meeting details
        title = Paragraph(meeting.get("title", "Untitled"), cell_style)
        location = Paragraph(meeting.get("location", "—"), cell_style)
        attendees = meeting.get("participants") or meeting.get("attendees") or []
        participants = Paragraph(", ".join(attendees) if isinstance(attendees, list) else str(attendees), cell_style)

        data.append([date, time, title, location, participants])

    # Create and style the table
    table = Table(data, colWidths=[3.5*cm, 3*cm, 5*cm, 4*cm, 5*cm])
    table.setStyle(TableStyle([
        # Header styling
        ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor("#6741d9")),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.white),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('ALIGN', (0, 0), (-1, 0), 'CENTER'),
        
        # Body styling
        ('FONTNAME', (0, 1), (-1, -1), 'Helvetica'),
        ('FONTSIZE', (0, 1), (-1, -1), 10),
        ('VALIGN', (0, 1), (-1, -1), 'TOP'),
        ('LEFTPADDING', (0, 1), (-1, -1), 6),
        ('RIGHTPADDING', (0, 1), (-1, -1), 6),
        ('BOTTOMPADDING', (0, 1), (-1, -1), 6),
        ('GRID', (0, 0), (-1, -1), 0.5, colors.grey),
        ('ROWBACKGROUNDS', (0, 1), (-1, -1), [colors.whitesmoke, colors.lightgrey])
    ]))

    elements.append(table)
    doc.build(elements)

    # Save final PDF to Downloads folder
    downloads_path = os.path.join(os.path.expanduser("~"), "Downloads")
    os.makedirs(downloads_path, exist_ok=True)

    filename = f"weekly_meeting_agenda_{datetime.now().strftime('%Y%m%d_%H%M%S')}.pdf"
    final_path = os.path.join(downloads_path, filename)
    shutil.copy(temp_pdf.name, final_path)

    return final_path

In [4]:
# Define the tool schema
local_tools = [{
    "declaration": {
        "type": "function",
        "function": {
            "name": "export_meeting_schedule_pdf",
            "description": "Generate a weekly meeting agenda as a PDF from a list of meetings.",
            "parameters": {
                "type": "object",
                "properties": {
                    "meetings": {
                        "type": "array",
                        "description": "List of meetings to include in the agenda.",
                        "items": {
                            "type": "object",
                            "properties": {
                                "title": {
                                    "type": "string",
                                    "description": "Title of the meeting"
                                },
                                "start_time": {
                                    "type": "string",
                                    "description": "Start time in ISO 8601 format"
                                },
                                "end_time": {
                                    "type": "string",
                                    "description": "End time in ISO 8601 format"
                                },
                                "location": {
                                    "type": "string",
                                    "description": "Meeting location (optional)"
                                },
                                "participants": {
                                    "type": "array",
                                    "items": {"type": "string"},
                                    "description": "List of participant names or emails"
                                }
                            },
                            "required": ["title", "start_time", "end_time"]
                        }
                    }
                },
                "required": ["meetings"]
            }
        }
    },
    "fn": export_meeting_schedule_pdf
}]

# Create tool mappings for the agent
local_tools_list = [tool['declaration'] for tool in local_tools]
local_tools_by_name = {tool['declaration']['function']['name']: tool['fn'] 
                       for tool in local_tools}

In [5]:
# Function to create or load an agent
def setup_agent() -> Agent:
    """load the meeting recording Agent from xpander.ai"""
    agent = xpander_client.agents.get(agent_id=os.environ['XPANDER_AGENT_ID'])
    print(f"🔄 Loaded agent: {agent.name}")
    print(f"🔍 View this agent in the Xpander platform with ID: https://app.xpander.ai/agents/{agent.id}")
    return agent

In [6]:
# Agent execution loop with local tools support
def agent_loop(agent: Agent, local_tools_by_name=None):
    print("🪄 Starting Agent Loop")
    # Initialize token tracking and timing
    execution_tokens = Tokens(worker=LLMTokens(completion_tokens=0, prompt_tokens=0, total_tokens=0))
    execution_start_time = time.perf_counter()
    
    while not agent.is_finished():
        # Get response from OpenAI
        start_time = time.perf_counter()
        response = openai_client.chat.completions.create(
            model="gpt-4o",
            messages=agent.messages,
            tools=agent.get_tools(llm_provider=LLMProvider.OPEN_AI),
            tool_choice=agent.tool_choice,
            temperature=0
        )
        
        # Track token usage
        execution_tokens.worker.completion_tokens += response.usage.completion_tokens
        execution_tokens.worker.prompt_tokens += response.usage.prompt_tokens
        execution_tokens.worker.total_tokens += response.usage.total_tokens
        
        # Report LLM usage to Xpander
        agent.report_llm_usage(
            llm_response=response.model_dump(),
            llm_inference_duration=time.perf_counter() - start_time,
            llm_provider=LLMProvider.OPEN_AI
        )
        
        agent.add_messages(response.model_dump())
        
        # Handle tool calls
        tool_calls = XpanderClient.extract_tool_calls(
            llm_response=response.model_dump(),
            llm_provider=LLMProvider.OPEN_AI
        )
        
        if tool_calls:
            # Display which tools are being used
            for call in tool_calls:
                name = getattr(call, 'name', None) or getattr(getattr(call, 'function', {}), 'name', "unnamed")
                print(f"🔧 Using tool: {name}")
                
            # Run cloud tools
            agent.run_tools(tool_calls=tool_calls)
            
            # Run local tools if provided
            if local_tools_by_name:
                pending_local_tool_execution = XpanderClient.retrieve_pending_local_tool_calls(tool_calls=tool_calls)
                local_tools_results = []
                
                for tc in pending_local_tool_execution:
                    print(f"🛠️ Running local tool: {tc.name}")
                    tool_call_result = ToolCallResult(function_name=tc.name, tool_call_id=tc.tool_call_id, payload=tc.payload)
                    try:
                        if tc.name in local_tools_by_name:
                            tool_call_result.is_success = True
                            tool_call_result.result = local_tools_by_name[tc.name](**tc.payload)
                        else:
                            raise Exception(f"Local tool {tc.name} not found")
                    except Exception as e:
                        tool_call_result.is_success = False
                        tool_call_result.is_error = True
                        tool_call_result.result = str(e)
                    finally:
                        local_tools_results.append(tool_call_result)

                if local_tools_results:
                    print(f"📝 Registering {len(local_tools_results)} local tool results...")
                    agent.memory.add_tool_call_results(tool_call_results=local_tools_results)
    
    # Report execution metrics to Xpander
    agent.report_execution_metrics(
        llm_tokens=execution_tokens,
        ai_model="gpt-4o"
    )
    
    print(f"✨ Execution duration: {time.perf_counter() - execution_start_time:.2f} seconds")
    print(f"🔢 Total tokens used: {execution_tokens.worker.total_tokens}")

In [7]:
def chat(agent: Agent, message, thread_id=None, local_tools_by_name=None):
    """Send a message to the agent and get a response"""
    print(f"\n👤 User: {message}")
    
    # Add task to agent (using thread_id for conversation continuity)
    agent.add_task(input=message, thread_id=thread_id)
    
    # Run the agent loop
    agent_loop(agent, local_tools_by_name)
    
    # Get and return result
    result = agent.retrieve_execution_result()
    print(f"🤖 Agent: {result.result}")
    print(f"🧵 Thread ID: {result.memory_thread_id}")
    return result.memory_thread_id

In [8]:
# initialize the agent
agent = setup_agent()      
# Add local tools to the agent
agent.add_local_tools(local_tools_list)
print("🧰 Local tools added to agent")
# start the conversation
thread_id = chat(agent, 'Hi! What can you do?', local_tools_by_name=local_tools_by_name)


🔄 Loaded agent: Meeting Recording Agent
🔍 View this agent in the Xpander platform with ID: https://app.xpander.ai/agents/8b5f4852-af17-4369-84bc-486af13cddbe
🧰 Local tools added to agent

👤 User: Hi! What can you do?
🪄 Starting Agent Loop
🔧 Using tool: xpfinish-agent-execution-finished
✨ Execution duration: 22.10 seconds
🔢 Total tokens used: 3098
🤖 Agent: Hello! I am a Meeting Automation Agent. I can help you with the following tasks:

1. **Record Meetings**: I can join your Google Meet or Zoom meetings using a recorder bot to capture the session.
2. **Transcribe Meetings**: After recording, I can transcribe the meeting to extract key topics, decisions, and action items.
3. **Summarize Meetings**: I can generate a structured summary of the meeting, highlighting important points and next steps.
4. **Send Follow-Up Emails**: I can send a synthesized email to meeting attendees with the summary of the meeting.
5. **Manage Calendar Events**: I can retrieve and manage your calendar events to

In [14]:
# adding a message to the existing thread and running the agentic loop
chat(agent, 'List my upcoming meetings on June 26, 2025 and the three consecutive days, for each meeting, include: title, description (if available), location, time, participants', thread_id, local_tools_by_name=local_tools_by_name)


👤 User: List my upcoming meetings on June 26, 2025 and the three consecutive days, for each meeting, include: title, description (if available), location, time, participants
🪄 Starting Agent Loop
🔧 Using tool: CalendarEventManagementGetCalendarEventsById
🔧 Using tool: export_meeting_schedule_pdf
🛠️ Running local tool: export_meeting_schedule_pdf
📝 Registering 1 local tool results...
🔧 Using tool: xpfinish-agent-execution-finished
✨ Execution duration: 36.64 seconds
🔢 Total tokens used: 13076
🤖 Agent: The meeting schedule for June 26, 2025, and the following three days has been successfully exported as a PDF. You can download it from the following link: [Download Meeting Schedule](C:\Users\jhaas\Downloads\weekly_meeting_agenda_20250627_165434.pdf).
🧵 Thread ID: 9aa62ed4-e62d-493d-ad9c-4c8d86dee411


'9aa62ed4-e62d-493d-ad9c-4c8d86dee411'

In [10]:
chat(agent, "Create meeting schedule for the upcoming 3 days, and export it as a PDF", thread_id, local_tools_by_name=local_tools_by_name)


👤 User: Create meeting schedule for the upcoming 3 days, and export it as a PDF
🪄 Starting Agent Loop
🔧 Using tool: CalendarEventManagementGetCalendarEventsById
🔧 Using tool: xpfinish-agent-execution-finished
✨ Execution duration: 28.63 seconds
🔢 Total tokens used: 6487
🤖 Agent: It seems there are no upcoming meetings scheduled in your calendar for the next three days. If you have any other requests or need further assistance, feel free to ask!
🧵 Thread ID: 9aa62ed4-e62d-493d-ad9c-4c8d86dee411


'9aa62ed4-e62d-493d-ad9c-4c8d86dee411'

In [11]:
# adding another message to the existing thread and running the agentic loop
chat(agent, 'Create a recorder for the <MEETING_TITLE>.', thread_id, local_tools_by_name=local_tools_by_name)


👤 User: Create a recorder for the <MEETING_TITLE>.
🪄 Starting Agent Loop
🔧 Using tool: xpfinish-agent-execution-finished
✨ Execution duration: 11.48 seconds
🔢 Total tokens used: 3349
🤖 Agent: I couldn't find any upcoming meetings in your calendar for the specified period. Please ensure that the meeting title is correct or provide more details so I can assist you further.
🧵 Thread ID: 9aa62ed4-e62d-493d-ad9c-4c8d86dee411


'9aa62ed4-e62d-493d-ad9c-4c8d86dee411'

In [12]:
chat(agent, 'Check the recorder status and give me the asset links if done.', thread_id, local_tools_by_name=local_tools_by_name)



👤 User: Check the recorder status and give me the asset links if done.
🪄 Starting Agent Loop
🔧 Using tool: xpfinish-agent-execution-finished
✨ Execution duration: 8.69 seconds
🔢 Total tokens used: 3377
🤖 Agent: I cannot proceed with creating a recorder or checking its status because I don't have the meeting details or the recorder ID. Please provide the meeting title or URL to create a recorder, or the recorder ID to check its status.
🧵 Thread ID: 9aa62ed4-e62d-493d-ad9c-4c8d86dee411


'9aa62ed4-e62d-493d-ad9c-4c8d86dee411'

In [13]:
chat(agent, 'Email the video & transcript to <YOUR_EMAIL> with a summary.', thread_id, local_tools_by_name=local_tools_by_name)


👤 User: Email the video & transcript to <YOUR_EMAIL> with a summary.
🪄 Starting Agent Loop
🔧 Using tool: xpfinish-agent-execution-finished
✨ Execution duration: 17.18 seconds
🔢 Total tokens used: 3403
🤖 Agent: I am unable to proceed with the tasks as there are no upcoming meetings found in the calendar for the specified period. Additionally, I cannot create a recorder or check its status without a specific meeting URL or title. Please provide more details or check your calendar settings.
🧵 Thread ID: 9aa62ed4-e62d-493d-ad9c-4c8d86dee411


'9aa62ed4-e62d-493d-ad9c-4c8d86dee411'